# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline


In [ ]:
import os
import json
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(override = True)
auth = os.getenv("auth")
groq_api_key = os.getenv("groq_api")
if groq_api_key and auth:
    print("Key is loaded!")
else:
    print("ERROR!")

In [ ]:
groq = OpenAI(base_url = "https://api.groq.com/openai/v1", api_key = groq_api_key)

In [ ]:
ticket_list = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def ticket_prices(destination_city):
    print(f"Tool called for {destination_city}")
    price = ticket_list.get(destination_city.lower(), "Not on this list!")
    return f"The ticket price for {destination_city} is {price}!"

In [ ]:
ticket_prices("kerala")

In [ ]:
system_message = f"""
You are a helpful assistant for an Airline called FlightAI.
Make sure you let the customer know that through introduciton.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
If they ask for guide or Menu or list for checking the prices of \
the destinations, just show them this list : - {ticket_list}
"""

In [ ]:
ticket_format = {
    "name": "ticket_prices",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }

        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": ticket_format}]
tools
type(tools)

In [ ]:
def tool_wheel(message):
    output = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "ticket_prices":
            args = json.loads(tool_call.function.arguments)
            d_city = args.get("destination_city")
            amount = ticket_prices(d_city)
            output.append(
                {
                    "role": "tool",
                    "content": amount,
                    "tool_call_id": tool_call.id
                }
                )
    return output

In [ ]:
def chat(message, history):
    history = [
        {"role": h["role"], "content": h["content"]} for h in history
        ]
    messages = [
        {"role": "system", "content": system_message}
        ] + history + [
            {"role": "user", "content": message}
            ] 
    response = groq.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        messages = messages,
        tools = tools
    )
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        answer = tool_wheel(message)
        messages.append(message)
        messages.extend(answer)
        break #Without it, your CPU will be eaten by list's extend method with the help of while loop. 
        #Check the following cell on difference between append(which Edward Donner used in the Udemy video) and extend
      
    flow = groq.chat.completions.create(
        model = "llama-3.3-70b-versatile",
        messages = messages,
        stream = True
    )

    reply = ""
    for chunk in flow:
        reply += chunk.choices[0].delta.content or ''
        yield reply


In [ ]:
### Difference between `append` and `extend` in Python lists

"""
list.append(x) ##adds x as a single element to the end of the list.
list.extend(iterable) ##iterates over iterable and adds each element to the list.
Examples:
lst = [1, 2]
lst.append([3, 4])   # -> [1, 2, [3, 4]]
lst = [1, 2]
lst.extend([3, 4])   # -> [1, 2, 3, 4]
"""
###Edge case with strings:
"""
lst = [1]
lst.append("ab")     # -> [1, "ab"]
lst = [1]
lst.extend("ab")     # -> [1, "a", "b"]
"""
###Both modify the list in-place and return None.


In [ ]:
#chat("hi", history = [])

In [ ]:
gr.ChatInterface(fn = chat, type = "messages").launch(inbrowser = True)#I'm using old version of gradio hence the type kwarg displayed
"""
Also use the below prompt for checking
'first pls check the price for London, and then check the price 4 Tokyo only if its below 1000'
"""